**LSTM without new**

In [1]:

import json
import time
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from numpy import newaxis
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from math import pi,sqrt,exp,pow,log
from numpy.linalg import det, inv
from abc import ABCMeta, abstractmethod
from sklearn import cluster

import statsmodels.api as sm
import scipy.stats as scs
import scipy.optimize as sco
import scipy.interpolate as sci
from scipy import stats

In [ ]:
import os
print(os.getcwd())

/content


In [7]:
split = (0.85);
sequence_length=10;
normalise= True
batch_size=100;
input_dim=1
input_timesteps=9
neurons=50
epochs=5
prediction_len=1
dense_output=1
drop_out=0

dataframe = pd.read_csv("/content/hpg.csv")
cols = ['Close']

len_dataframe=dataframe.shape[0]

In [8]:
# Loại bỏ các hàng chứa NaN
dataframe = dataframe.dropna()

# Hoặc nếu bạn muốn loại bỏ các cột chứa NaN:
dataframe = dataframe.dropna(axis=1)

# Lưu lại file CSV sau khi loại bỏ NaN
dataframe.to_csv("/content/hpg_cleaned.csv", index=False)

# Kiểm tra lại dữ liệu sau khi loại bỏ NaN
print(dataframe.head())

   Unnamed: 0       Date  Close  cafef_mean_compound  vietstock_mean_compound  \
0         0.0  31/7/2024   27.0             0.212000                  0.12440   
1         1.0  30/7/2024   28.0             0.000000                 -0.13660   
2         2.0  29/7/2024   28.0             0.074667                  0.15910   
3         3.0  26/7/2024   28.0             0.435667                 -0.15910   
4         4.0  25/7/2024   27.0            -0.167433                  0.62885   

   economy_mean_compound  ktck_mean_compound  
0               0.098667            0.148000  
1               0.403300            0.064125  
2               0.230825            0.083733  
3               0.217200           -0.120400  
4               0.415650            0.058475  


In [9]:
print(dataframe.columns)


Index(['Unnamed: 0', 'Date', 'Close', 'cafef_mean_compound',
       'vietstock_mean_compound', 'economy_mean_compound',
       'ktck_mean_compound'],
      dtype='object')


In [10]:
i_split = int(len(dataframe) * split)
data_train = dataframe.get(cols).values[:i_split]
data_test  = dataframe.get(cols).values[i_split:]
len_train  = len(data_train)
len_test   = len(data_test)
len_train_windows = None
print('data_train.shape',data_train.shape)
print('data_test.shape',data_test.shape)

data_train.shape (120, 1)
data_test.shape (22, 1)


In [11]:
#get_test_data
data_windows = []
for i in range(len_test - sequence_length):
    data_windows.append(data_test[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)
 # get original y_test
y_test_ori = data_windows[:, -1, [0]]
print('y_test_ori.shape',y_test_ori.shape)

window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]
normalised_data = []
record_min=[]
record_max=[]

y_test_ori.shape (12, 1)


In [12]:
#normalize
for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)


<ipython-input-12-82761ccc053e>:13: RuntimeWarning: invalid value encountered in divide
  temp_col=temp_col/temp_max


In [13]:
# normalised_data=window_data
data_windows=normalised_data#get_test_data
x_test = data_windows[:, :-1]
y_test = data_windows[:, -1, [0]]
print('x_test.shape',x_test.shape)
print('y_test.shape',y_test.shape)


x_test.shape (12, 9, 1)
y_test.shape (12, 1)


In [14]:
#get_train_data
data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)

window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []
for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)


<ipython-input-14-bc750d2c525c>:19: RuntimeWarning: invalid value encountered in divide
  temp_col=temp_col/temp_max


In [15]:
# normalised_data=window_data
data_windows=normalised_data
x_train = data_windows[:, :-1]
y_train = data_windows[:, -1,[0]]
print('x_train.shape',x_train.shape)
print('y_train.shape',y_train.shape)


x_train.shape (110, 9, 1)
y_train.shape (110, 1)


In [16]:
import numpy as np
print(np.isnan(x_train).sum(), np.isinf(x_train).sum())
print(np.isnan(y_train).sum(), np.isinf(y_train).sum())


18 0
2 0


In [17]:
x_train = np.nan_to_num(x_train)
y_train = np.nan_to_num(y_train)

In [18]:
# LSTM MODEL
model = Sequential()
model.add(LSTM(neurons, input_shape=(input_timesteps, input_dim), return_sequences = True))
model.add(Dropout(drop_out))
model.add(LSTM(neurons,return_sequences = True))
model.add(LSTM(neurons,return_sequences =False))
model.add(Dropout(drop_out))
model.add(Dense(dense_output, activation='linear'))
model.compile(loss='mean_squared_error',
                optimizer='adam')
# Fit the model
model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size)


Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 79ms/step - loss: 0.5463
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.4887 
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.4381
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.3881
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.3325


In [19]:
import numpy as np
print(np.isnan(x_test).sum(), np.isinf(x_test).sum())
print(np.isnan(y_test).sum(), np.isinf(y_test).sum())

36 0
4 0


In [20]:
x_test = np.nan_to_num(x_test)
y_test = np.nan_to_num(y_test)

In [21]:
model.predict(x_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 628ms/step


array([[0.39829427],
       [0.4369645 ],
       [0.41633692],
       [0.38694668],
       [0.34959567],
       [0.30574182],
       [0.25728285],
       [0.20635226],
       [0.1551296 ],
       [0.1551296 ],
       [0.1551296 ],
       [0.1551296 ]], dtype=float32)

In [22]:
#multi sequence predict
data=x_test
prediction_seqs = []
window_size=sequence_length
pre_win_num=int(len(data)/prediction_len)

for i in range(0,pre_win_num):
    curr_frame = data[i*prediction_len]
    predicted = []
    for j in range(0,prediction_len):
      temp=model.predict(curr_frame[newaxis,:,:])[0]
      predicted.append(temp)
      curr_frame = curr_frame[1:]
      curr_frame = np.insert(curr_frame, [window_size-2], predicted[-1], axis=0)
    prediction_seqs.append(predicted)

print(prediction_seqs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[array([0.3982942], dtype=float32)], [array([0.43696454], dtype=float32)], [array([0.41633692], dtype=float32)], [array([0.38694668], dtype=float32)], [array([0.3495956], dtype=float32)], [array([0.30574176], dtype=float32)], [array([0.25728288], dtype=float32)], [array([0.20635223], dtype=float32)], [array([0.15512963], dtype=float32)], [array([0.15512963], dtype=float32)], [array([0.15512963], dtype=float32)], [array([0.15512963], dtype=float32)]]


In [23]:
#de_predicted
de_predicted=[]
len_pre_win=int(len(data)/prediction_len)
len_pre=prediction_len

m=0
for i in range(0,len_pre_win):
    for j in range(0,len_pre):
      de_predicted.append(prediction_seqs[i][j][0]*record_max[m]+record_min[m])
      m=m+1
print(de_predicted)



[25.39829421043396, 25.43696454167366, 25.41633692383766, 25.38694667816162, 25.349595606327057, 25.305741757154465, 25.25728288292885, 25.20635223388672, 25.0, 25.0, 25.0, 25.0]


In [24]:
error = []
diff=y_test.shape[0]-prediction_len*pre_win_num

for i in range(y_test_ori.shape[0]-diff):
    error.append(y_test_ori[i,] - de_predicted[i])

squaredError = []
absError = []
for val in error:
    squaredError.append(val * val)
    absError.append(abs(val))

error_percent=[]
for i in range(len(error)):
    val=absError[i]/y_test_ori[i,]
    val=abs(val)
    error_percent.append(val)

mean_error_percent=sum(error_percent) / len(error_percent)
accuracy=1-mean_error_percent

MSE=sum(squaredError) / len(squaredError)

print("MSE",MSE)
print('accuracy',accuracy)
print('mean_error_percent',mean_error_percent)

MSE [0.08309261]
accuracy [0.99080828]
mean_error_percent [0.00919172]


**LSTM with new**

In [26]:
import json
import time
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from numpy import newaxis
from keras.layers import Dense, Activation, Dropout, LSTM, Input
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from math import pi,sqrt,exp,pow,log
from numpy.linalg import det, inv
from abc import ABCMeta, abstractmethod
from sklearn import cluster

import statsmodels.api as sm
import scipy.stats as scs
import scipy.optimize as sco
import scipy.interpolate as sci
from scipy import stats


split = (0.85);
sequence_length=10;
normalise= True
batch_size=100;
input_dim=5
input_timesteps=9
neurons=50
epochs=5
prediction_len=1
dense_output=1
drop_out=0

dataframe = pd.read_csv("/content/hpg_cleaned.csv")
print(dataframe.columns)

Index(['Unnamed: 0', 'Date', 'Close', 'cafef_mean_compound',
       'vietstock_mean_compound', 'economy_mean_compound',
       'ktck_mean_compound'],
      dtype='object')


In [28]:
cols = ['Close','cafef_mean_compound',
       'vietstock_mean_compound', 'economy_mean_compound',
       'ktck_mean_compound']

len_dataframe=dataframe.shape[0]

i_split = int(len(dataframe) * split)
data_train = dataframe.get(cols).values[:i_split]
data_test  = dataframe.get(cols).values[i_split:]
len_train  = len(data_train)
len_test   = len(data_test)
len_train_windows = None
print('data_train.shape',data_train.shape)
print('data_test.shape',data_test.shape)


data_train.shape (120, 5)
data_test.shape (22, 5)


In [29]:

data_train[0:5]

array([[27.        ,  0.212     ,  0.1244    ,  0.09866667,  0.148     ],
       [28.        ,  0.        , -0.1366    ,  0.4033    ,  0.064125  ],
       [28.        ,  0.07466667,  0.1591    ,  0.230825  ,  0.08373333],
       [28.        ,  0.43566667, -0.1591    ,  0.2172    , -0.1204    ],
       [27.        , -0.16743333,  0.62885   ,  0.41565   ,  0.058475  ]])

In [30]:
data_test[0:5]

array([[ 2.50000000e+01,  1.48000000e-01,  2.91866667e-01,
         2.86000000e-02,  8.44200000e-01],
       [ 2.60000000e+01,  5.26700000e-01,  1.06066667e-01,
        -8.50333330e-02,  2.36700000e-01],
       [ 2.60000000e+01,  0.00000000e+00, -2.55500000e-02,
         1.61600000e-01,  2.12000000e-01],
       [ 2.60000000e+01,  0.00000000e+00,  2.25933333e-01,
         3.74333333e-01, -8.60000000e-03],
       [ 2.60000000e+01,  0.00000000e+00, -1.46666670e-02,
         1.80800000e-01,  2.32633333e-01]])

In [31]:
data_windows = []
for i in range(len_test - sequence_length):
    data_windows.append(data_test[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)
 # get original y_test
y_test_ori = data_windows[:, -1, [0]]
print('y_test_ori.shape',y_test_ori.shape)

y_test_ori.shape (12, 1)


In [32]:
window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]
normalised_data = []
record_min=[]
record_max=[]

In [33]:
#normalize
for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data#get_test_data
x_test = data_windows[:, :-1]
y_test = data_windows[:, -1, [0]]
print('x_test.shape',x_test.shape)
print('y_test.shape',y_test.shape)

x_test.shape (12, 9, 5)
y_test.shape (12, 1)


<ipython-input-33-5c0a7a42e98d>:13: RuntimeWarning: invalid value encountered in divide
  temp_col=temp_col/temp_max


In [34]:
x_test[0]

array([[ 0.        ,  0.148     ,  0.29186667,  0.0286    ,  0.8442    ],
       [ 1.        ,  0.5267    ,  0.10606667, -0.08503333,  0.2367    ],
       [ 1.        ,  0.        , -0.02555   ,  0.1616    ,  0.212     ],
       [ 1.        ,  0.        ,  0.22593333,  0.37433333, -0.0086    ],
       [ 1.        ,  0.        , -0.01466667,  0.1808    ,  0.23263333],
       [ 1.        ,  0.4144    ,  0.4378    ,  0.26013333, -0.21823333],
       [ 1.        ,  0.2294    , -0.0334    ,  0.0911    ,  0.148     ],
       [ 1.        ,  0.2997    , -0.17      ,  0.17963333,  0.2027    ],
       [ 0.        ,  0.25825   , -0.1591    , -0.2276    , -0.0593    ]])

In [35]:
y_test

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [nan],
       [nan],
       [nan],
       [nan]])

In [36]:
#get_train_data
data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)

window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []

for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data
x_train = data_windows[:, :-1]
y_train = data_windows[:, -1,[0]]
print('x_train.shape',x_train.shape)
print('y_train.shape',y_train.shape)

x_train.shape (110, 9, 5)
y_train.shape (110, 1)


<ipython-input-36-05552b826d82>:24: RuntimeWarning: invalid value encountered in divide
  temp_col=temp_col/temp_max


In [37]:
x_train[0]

array([[ 0.        ,  0.212     ,  0.1244    ,  0.09866667,  0.148     ],
       [ 1.        ,  0.        , -0.1366    ,  0.4033    ,  0.064125  ],
       [ 1.        ,  0.07466667,  0.1591    ,  0.230825  ,  0.08373333],
       [ 1.        ,  0.43566667, -0.1591    ,  0.2172    , -0.1204    ],
       [ 0.        , -0.16743333,  0.62885   ,  0.41565   ,  0.058475  ],
       [ 1.        ,  0.29086667,  0.        ,  0.2884    ,  0.        ],
       [ 1.        ,  0.53505   ,  0.        ,  0.35775   ,  0.06906667],
       [ 1.        ,  0.3803    ,  0.2787    ,  0.2971    , -0.33525   ],
       [ 1.        ,  0.26026667, -0.06005   ,  0.318     ,  0.2732    ]])

In [38]:
y_train[0:5]

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [39]:
import numpy as np
print(np.isnan(x_train).sum(), np.isinf(x_train).sum())
print(np.isnan(y_train).sum(), np.isinf(y_train).sum())
x_train = np.nan_to_num(x_train)
y_train = np.nan_to_num(y_train)

18 0
2 0


In [40]:
x_train = np.nan_to_num(x_train)
y_train = np.nan_to_num(y_train)


In [41]:
import numpy as np
print(np.isnan(x_test).sum(), np.isinf(x_test).sum())
print(np.isnan(y_test).sum(), np.isinf(y_test).sum())
x_test = np.nan_to_num(x_test)
y_test = np.nan_to_num(y_test)

36 0
4 0


In [42]:
# LSTM MODEL
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Input
model = Sequential()
model.add(Input(shape=(input_timesteps, input_dim)))
model.add(LSTM(neurons, input_shape=(input_timesteps, input_dim), return_sequences = True))
model.add(Dropout(drop_out))
model.add(LSTM(neurons,return_sequences = True))
model.add(LSTM(neurons,return_sequences =False))
model.add(Dropout(drop_out))
model.add(Dense(dense_output, activation='linear'))
# Compile model
model.compile(loss='mean_squared_error',
                optimizer='adam')
# Fit the model
model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size)



Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - loss: 0.5237
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.4436
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.3874
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.3137
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.2625


In [43]:
# LSTM MODEL
model = Sequential()
model.add(LSTM(neurons, input_shape=(input_timesteps, input_dim), return_sequences = True))
model.add(Dropout(drop_out))
model.add(LSTM(neurons,return_sequences = True))
model.add(LSTM(neurons,return_sequences =False))
model.add(Dropout(drop_out))
model.add(Dense(dense_output, activation='linear'))
# Compile model
model.compile(loss='mean_squared_error',
                optimizer='adam')
# Fit the model
model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size)


model.predict(x_test)


#multi sequence predict
data=x_test
prediction_seqs = []
window_size=sequence_length
pre_win_num=int(len(data)/prediction_len)

for i in range(0,pre_win_num):
    curr_frame = data[i*prediction_len]
    predicted = []
    for j in range(0,prediction_len):
      temp=model.predict(curr_frame[newaxis,:,:])[0]
      predicted.append(temp)
      curr_frame = curr_frame[1:]
      curr_frame = np.insert(curr_frame, [window_size-2], predicted[-1], axis=0)
    prediction_seqs.append(predicted)

print(prediction_seqs)


#de_predicted
de_predicted=[]
len_pre_win=int(len(data)/prediction_len)
len_pre=prediction_len

m=0
for i in range(0,len_pre_win):
    for j in range(0,len_pre):
      de_predicted.append(prediction_seqs[i][j][0]*record_max[m]+record_min[m])
      m=m+1
print(de_predicted)

error = []
diff=y_test.shape[0]-prediction_len*pre_win_num

for i in range(y_test_ori.shape[0]-diff):
    error.append(y_test_ori[i,] - de_predicted[i])

squaredError = []
absError = []
for val in error:
    squaredError.append(val * val)
    absError.append(abs(val))

error_percent=[]
for i in range(len(error)):
    val=absError[i]/y_test_ori[i,]
    val=abs(val)
    error_percent.append(val)

mean_error_percent=sum(error_percent) / len(error_percent)
accuracy=1-mean_error_percent

MSE=sum(squaredError) / len(squaredError)

print("MSE",MSE)
print('accuracy',accuracy)
print('mean_error_percent',mean_error_percent)



Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 0.5494
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.4783 
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.3986
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.3403
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.2751
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 437ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
[[array([0.55978656], dtype=float32)], [array([0.5977171], dtype=float32)], [array([0.5565465], dtype=float32)], [array([0.5331179], dtype=float32)], [array([0.5065663], dtype=float32)], [

**DP-LSTM**

In [45]:
import json
import time
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from numpy import newaxis
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from math import pi,sqrt,exp,pow,log
from numpy.linalg import det, inv
from abc import ABCMeta, abstractmethod
from sklearn import cluster

import statsmodels.api as sm
import scipy.stats as scs
import scipy.optimize as sco
import scipy.interpolate as sci
from scipy import stats


df = pd.read_csv("/content/hpg_cleaned.csv")

cafef_var=np.var(df.cafef_mean_compound)
vietstock_var=np.var(df.vietstock_mean_compound)
economy_var=np.var(df.economy_mean_compound)
ktck_var=np.var(df.ktck_mean_compound)

mu=0

noise=0.1

sigma_cafef=noise*cafef_var
sigma_vietstock=noise*vietstock_var
sigma_economy=noise*economy_var
sigma_ktck=noise*ktck_var

n=df.shape[0]

df_noise=pd.DataFrame()

df_noise['cafef_noise']=df['cafef_mean_compound']
df_noise['vietstock_noise']=df['vietstock_mean_compound']
df_noise['economy_noise']=df['economy_mean_compound']
df_noise['ktck_noise']=df['ktck_mean_compound']

for i in range(0,n):
    df_noise['cafef_noise'][i]+=np.random.normal(mu,sigma_cafef)
    df_noise['vietstock_noise'][i]+=np.random.normal(mu,sigma_vietstock)
    df_noise['economy_noise'][i]+=np.random.normal(mu,sigma_economy)
    df_noise['ktck_noise'][i]+=np.random.normal(mu,sigma_ktck)


df_noise.to_csv("/content/source_price_noise0.csv")
dfn=pd.read_csv("/content/source_price_noise0.csv",index_col=0)

In [48]:
df_1n=pd.DataFrame()
df_1n['cafef']=df_noise['cafef_noise']
df_1n['vietstock']=df['vietstock_mean_compound']
df_1n['economy']=df['economy_mean_compound']
df_1n['ktck']=df['ktck_mean_compound']
df_1n['price']=df['Close']

df_2n=pd.DataFrame()
df_2n['cafef']=df['cafef_mean_compound']
df_2n['vietstock']=df_noise['vietstock_noise']
df_2n['economy']=df['economy_mean_compound']
df_2n['ktck']=df['ktck_mean_compound']
df_2n['price']=df['Close']

df_3n=pd.DataFrame()
df_3n['cafef']=df['cafef_mean_compound']
df_3n['vietstock']=df['vietstock_mean_compound']
df_3n['economy']=df_noise['economy_noise']
df_3n['ktck']=df['ktck_mean_compound']
df_3n['price']=df['Close']

df_4n=pd.DataFrame()
df_4n['cafef']=df['cafef_mean_compound']
df_4n['vietstock']=df['vietstock_mean_compound']
df_4n['economy']=df['economy_mean_compound']
df_4n['ktck']=df_noise['ktck_noise']
df_4n['price']=df['Close']

df1=df_1n
df2=df_2n
df3=df_3n
df4=df_4n

In [49]:

split = (0.85)
sequence_length=10;
normalise= True
batch_size=100;
input_dim=5
input_timesteps=9
neurons=50
epochs=5
prediction_len=1
dense_output=1
drop_out=0



i_split = int(len(df1) * split)

cols = ['price','cafef','vietstock','economy','ktck']
data_train_1 = df1.get(cols).values[:i_split]
data_train_2 = df2.get(cols).values[:i_split]
data_train_3 = df3.get(cols).values[:i_split]
data_train_4 = df4.get(cols).values[:i_split]

len_train  = len(data_train_1)
len_train_windows = None



#get_train_data
data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train_1[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)

window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []
record_min=[]
record_max=[]

for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data
x_train1 = data_windows[:, :-1]
y_train1 = data_windows[:, -1,[0]]
print('x_train1.shape',x_train1.shape)
print('y_train1.shape',y_train1.shape)


x_train1.shape (110, 9, 5)
y_train1.shape (110, 1)


<ipython-input-49-3101cfe47b46>:53: RuntimeWarning: invalid value encountered in divide
  temp_col=temp_col/temp_max


In [50]:
#get_train_data
data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train_2[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)

window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []
record_min=[]
record_max=[]

for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data
x_train2 = data_windows[:, :-1]
y_train2 = data_windows[:, -1,[0]]
print('x_train2.shape',x_train2.shape)
print('y_train2.shape',y_train2.shape)


x_train2.shape (110, 9, 5)
y_train2.shape (110, 1)


<ipython-input-50-7496be7d3366>:26: RuntimeWarning: invalid value encountered in divide
  temp_col=temp_col/temp_max


In [51]:
#get_train_data
data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train_3[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)

window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []
record_min=[]
record_max=[]

for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data
x_train3 = data_windows[:, :-1]
y_train3 = data_windows[:, -1,[0]]
print('x_train3.shape',x_train3.shape)
print('y_train3.shape',y_train3.shape)

x_train3.shape (110, 9, 5)
y_train3.shape (110, 1)


<ipython-input-51-62d2563059fd>:26: RuntimeWarning: invalid value encountered in divide
  temp_col=temp_col/temp_max


In [52]:

#get_train_data

data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train_4[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)

window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []
record_min=[]
record_max=[]

for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data
x_train4 = data_windows[:, :-1]
y_train4 = data_windows[:, -1,[0]]
print('x_train4.shape',x_train4.shape)
print('y_train4.shape',y_train4.shape)

x_train4.shape (110, 9, 5)
y_train4.shape (110, 1)


<ipython-input-52-e3f11bfd4ae9>:27: RuntimeWarning: invalid value encountered in divide
  temp_col=temp_col/temp_max


In [53]:

##concat train window
type(x_train4)


numpy.ndarray

In [54]:

x_train_t=np.concatenate((x_train1,x_train2,x_train3,x_train4),axis=0)
print(x_train_t.shape)
x_train=x_train_t

y_train_t=np.concatenate((y_train1,y_train2,y_train3,y_train4),axis=0)
print(y_train_t.shape)
y_train=y_train_t


(440, 9, 5)
(440, 1)


In [56]:
dataframe = pd.read_csv("/content/hpg_cleaned.csv")
print(dataframe.columns)  # Check the actual number of columns

Index(['Unnamed: 0', 'Date', 'Close', 'cafef_mean_compound',
       'vietstock_mean_compound', 'economy_mean_compound',
       'ktck_mean_compound'],
      dtype='object')


In [59]:
dataframe= pd.read_csv("/content/hpg_cleaned.csv")
dataframe.columns=['Unnamed: 0', 'Date', 'Close', 'cafef_mean_compound',
       'vietstock_mean_compound', 'economy_mean_compound',
       'ktck_mean_compound']
cols = ['Close', 'cafef_mean_compound',
       'vietstock_mean_compound', 'economy_mean_compound',
       'ktck_mean_compound']
len_dataframe=dataframe.shape[0]

In [60]:

len_dataframe=dataframe.shape[0]

i_split = int(len(dataframe) * split)
data_test  = dataframe.get(cols).values[i_split:]

len_test   = len(data_test)
len_train_windows = None

print('data_test.shape',data_test.shape)

data_test.shape (22, 5)


In [61]:
data_windows = []
for i in range(len_test - sequence_length):
    data_windows.append(data_test[i:i+sequence_length])

data_windows = np.array(data_windows).astype(float)

y_test_ori = data_windows[:, -1, [0]]
print('y_test_ori.shape',y_test_ori.shape)


y_test_ori.shape (12, 1)


In [62]:
y_test_ori

array([[25.],
       [25.],
       [25.],
       [25.],
       [25.],
       [25.],
       [25.],
       [25.],
       [25.],
       [25.],
       [25.],
       [25.]])

In [63]:
window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]
normalised_data = []
record_min=[]
record_max=[]

#normalize
for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data#get_test_data
x_test = data_windows[:, :-1]
y_test = data_windows[:, -1, [0]]
print('x_test.shape',x_test.shape)
print('y_test.shape',y_test.shape)


x_test.shape (12, 9, 5)
y_test.shape (12, 1)


<ipython-input-63-0543c195df03>:20: RuntimeWarning: invalid value encountered in divide
  temp_col=temp_col/temp_max


In [64]:
import numpy as np
print(np.isnan(x_train).sum(), np.isinf(x_train).sum())
print(np.isnan(y_train).sum(), np.isinf(y_train).sum())

72 0
8 0


In [65]:
x_train = np.nan_to_num(x_train)
y_train = np.nan_to_num(y_train)

In [66]:
import numpy as np
print(np.isnan(x_test).sum(), np.isinf(x_test).sum())
print(np.isnan(y_test).sum(), np.isinf(y_test).sum())

36 0
4 0


In [67]:
x_test = np.nan_to_num(x_test)
y_test = np.nan_to_num(y_test)

In [68]:
# LSTM MODEL
model = Sequential()
model.add(LSTM(neurons, input_shape=(input_timesteps, input_dim), return_sequences = True))
model.add(Dropout(drop_out))
model.add(LSTM(neurons,return_sequences = True))
model.add(LSTM(neurons,return_sequences =False))
model.add(Dropout(drop_out))
model.add(Dense(dense_output, activation='linear'))
# Compile model
model.compile(loss='mean_squared_error',
                optimizer='adam')
# Fit the model
model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 0.4691
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.3259
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.2615
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.2559
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.2360


In [69]:
#multi sequence predict
data=x_test
prediction_seqs = []
window_size=sequence_length
pre_win_num=int(len(data)/prediction_len)

for i in range(0,pre_win_num):
    curr_frame = data[i*prediction_len]
    predicted = []
    for j in range(0,prediction_len):
      temp=model.predict(curr_frame[newaxis,:,:])[0]
      predicted.append(temp)
      curr_frame = curr_frame[1:]
      curr_frame = np.insert(curr_frame, [window_size-2], predicted[-1], axis=0)
    prediction_seqs.append(predicted)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 440ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


In [70]:

#de_predicted
de_predicted=[]
len_pre_win=int(len(data)/prediction_len)
len_pre=prediction_len

m=0
for i in range(0,len_pre_win):
    for j in range(0,len_pre):
      de_predicted.append(prediction_seqs[i][j][0]*record_max[m]+record_min[m])
      m=m+1
print(de_predicted)

[25.518074452877045, 25.538781106472015, 25.507570505142212, 25.489556521177292, 25.457850754261017, 25.429786443710327, 25.36895713210106, 25.32721099257469, 25.0, 25.0, 25.0, 25.0]


In [71]:
 np.save('/content/sp_5dim_n01_7034.npy',de_predicted)

In [73]:
error = []
diff=y_test.shape[0]-prediction_len*pre_win_num

for i in range(y_test_ori.shape[0]-diff):
    error.append(y_test_ori[i,] - de_predicted[i])

squaredError = []
absError = []
for val in error:
    squaredError.append(val * val)
    absError.append(abs(val))

error_percent=[]
for i in range(len(error)):
    val=absError[i]/y_test_ori[i,]
    val=abs(val)
    error_percent.append(val)

mean_error_percent=sum(error_percent) / len(error_percent)
accuracy=1-mean_error_percent

MSE=sum(squaredError) / len(squaredError)

print("MSE",MSE)
print('accuracy',accuracy)
print('mean_error_percent',mean_error_percent)



MSE [0.14112664]
accuracy [0.98787404]
mean_error_percent [0.01212596]
